In [1]:
import os
os.environ['CURL_CA_BUNDLE'] = ""

In [2]:
import requests
requests.__version__

'2.27.0'

In [3]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --user

In [4]:
# !pip install transformers

In [5]:
from accelerate import Accelerator
import re

In [6]:
# !pip install transformers accelerate

from transformers import AutoTokenizer
import transformers
import torch

In [7]:
import torch

In [8]:
import os
os.environ['CURL_CA_BUNDLE'] = ""

In [9]:
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.agents import initialize_agent
from langchain.agents import Tool

from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [10]:
from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By

In [12]:
import langchain
langchain.__version__

'0.0.330'

In [13]:
import chromadb
chromadb.__version__

'0.4.15'

In [ ]:
n_gpu_layers = 1   # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=r"llama-2-7b-chat.Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4000,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
#     verbose=False,
    temperature=0.1,
    device=0,
    streaming=False
)

In [15]:
accelerator = Accelerator()
llm = accelerator.prepare(llm)

In [16]:
# llm("Simulate a rap battle between Stephen Colbert and John Oliver")

In [17]:
class BlogPosts(BaseModel):
    """Provides list of all the relevant blog posts and links based on the user query input."""
    query: str = Field(description="list of key words to search the blogpost for.")

def get_post_names(query):
    """Provides list of all the relevant blog posts and links based on the user query input.
    Output can have more than one post titles."""

    # striping off unwanted text if any
    query = query.replace('search terms', '')
    query = query.replace('keywords', '')
    query = query.replace(':', '')
    query = query.replace("'", '')

    posts=[]
    driver = webdriver.Chrome() # Update this with your chromedriver path
    blog_url = "https://sandeeprshah.blogspot.com/"
    blog_url = blog_url+"search?q="+query
    # Open the blog website in the browser
    driver.get(blog_url)

    time.sleep(5)  # Wait for 5 seconds to let the page load

    # Find all the search results (assuming search results are represented as links)
    search_results = driver.find_elements(By.CSS_SELECTOR, 'h3')

    # Extract and print the titles of the search results
    print("Titles and links of blog posts:")
    for result in search_results:
        post_title = result.text
        post_link = result.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#         print(f"Title: {post_title}")
#         print(f"Link: {post_link}")
#         print()
        post_title_link = post_title+":"+post_link
        posts.append(post_title)

    # Close the browser
    driver.quit()

    return {"response": posts}

In [18]:
get_post_names_tool = Tool(
    name="get_posts",
    func=get_post_names,
    description="Provides list of all the relevant blog posts and the links based on the user query input.",
    args_schema = BlogPosts
)

In [19]:
class SinglePost(BaseModel):
    """Given a blog title - find and open the blog in new browser window."""
    link: str = Field(description="link of the blogpost to open in new window")

def open_blog_post_new_window(link):
    """given link for a blog article - open it in new window"""

    # striping off unwanted text if any
    link = link.replace('search terms', '')
    link = link.replace("'", '')

    posts=[]
    driver = webdriver.Chrome() # Update this with your chromedriver path

    driver.get(link)

    time.sleep(5)  # Wait for 5 seconds to let the page load


    return {"response": "Post opened"}

In [20]:
open_blog_post_tool = Tool(
    name="open_blog_post_new_window",
    func=open_blog_post_new_window,
    description="given link for a blog article - open it in new window",
    args_schema = SinglePost
)

In [21]:
tools = [
    get_post_names_tool,
    open_blog_post_tool
]

In [22]:
PREFIX = "<<SYS>> You are smart that selects a function from list of functions based on user queries.\
Run only one function tool at a time or in one query.<</SYS>>\n"

In [23]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations = 4,
    agent_kwargs={
        'prefix': PREFIX, 
#         'format_instructions': FORMAT_INSTRUCTIONS,
#         'suffix': SUFFIX
    }
)

In [24]:
print(agent.agent.llm_chain.prompt.template)

<<SYS>> You are smart that selects a function from list of functions based on user queries.Run only one function tool at a time or in one query.<</SYS>>


get_posts: Provides list of all the relevant blog posts and the links based on the user query input.
open_blog_post_new_window: given link for a blog article - open it in new window

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get_posts, open_blog_post_new_window]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [25]:
A = agent.run("Fetch me titles of blog posts relevant to keywords: 'marathon training'. \
I just want the list of titles.\
DO NOT OPEN ANY LINKS.")



> Entering new AgentExecutor chain...



llama_print_timings:        load time =    7212.18 ms
llama_print_timings:      sample time =       4.32 ms /    36 runs   (    0.12 ms per token,  8341.06 tokens per second)
llama_print_timings: prompt eval time =    7212.02 ms /   249 tokens (   28.96 ms per token,    34.53 tokens per second)
llama_print_timings:        eval time =    5378.44 ms /    35 runs   (  153.67 ms per token,     6.51 tokens per second)
llama_print_timings:       total time =   12657.54 ms /   284 tokens


 Hmm, interesting! Let me check what I can find for you.
Action: get_posts
Action Input: keywords: 'marathon training'Titles and links of blog posts:

Observation: {'response': ['Reflecting on my marathon journey', 'Ticket to Standard Charted Mumbai Marathon (SCMM)', 'First Half Marathon + 2015 Resolutions', 'Critical Last Month', 'B2B - Back to Basics', 'My Cookie Jar', 'A decade of transition', 'New 30-day challenge', 'Day 6 - 30 Day Miracle Morning Challenge', 'When you get more than you asked for - 2023 !!', 'Reflecting on First Half of 2015', 'MoM - Man on Mission', 'The First Juggling Performance of My LIFE', 'Breaking Barriers !!!', 'Moving out of Comfort Zone', 'Day 10 - 30 Day Miracle Morning Challenge', 'Day 11 - 30 Day Miracle Morning Challenge', "Sandy's top 10 movie picks"]}
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7212.18 ms
llama_print_timings:      sample time =       5.68 ms /    50 runs   (    0.11 ms per token,  8799.72 tokens per second)
llama_print_timings: prompt eval time =    2077.82 ms /   218 tokens (    9.53 ms per token,   104.92 tokens per second)
llama_print_timings:        eval time =    7520.65 ms /    49 runs   (  153.48 ms per token,     6.52 tokens per second)
llama_print_timings:       total time =    9685.65 ms /   267 tokens


 Hmm, interesting! Let me check what I can find for you.
Final Answer: {'response': ['Reflecting on my marathon journey', 'Ticket to Standard Charted Mumbai Marathon (SCMM)']}

> Finished chain.


In [26]:
A

"{'response': ['Reflecting on my marathon journey', 'Ticket to Standard Charted Mumbai Marathon (SCMM)']}"

In [27]:
link_test = "https://sandeeprshah.blogspot.com/2023/01/reflecting-on-my-marathon-journey.html"

In [28]:
A = agent.run("Open blog post article with following link - '"+link_test+"'")

Llama.generate: prefix-match hit




> Entering new AgentExecutor chain...



llama_print_timings:        load time =    7212.18 ms
llama_print_timings:      sample time =       8.18 ms /    72 runs   (    0.11 ms per token,  8801.96 tokens per second)
llama_print_timings: prompt eval time =    1350.46 ms /    51 tokens (   26.48 ms per token,    37.77 tokens per second)
llama_print_timings:        eval time =   11029.84 ms /    71 runs   (  155.35 ms per token,     6.44 tokens per second)
llama_print_timings:       total time =   12507.64 ms /   122 tokens


 Given the link, I should open it in new window.
Action: open_blog_post_new_window
Action Input: 'https://sandeeprshah.blogspot.com/2023/01/reflecting-on-my-marathon-journey.html'
Observation: {'response': 'Post opened'}
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7212.18 ms
llama_print_timings:      sample time =       2.61 ms /    23 runs   (    0.11 ms per token,  8798.78 tokens per second)
llama_print_timings: prompt eval time =     708.15 ms /    12 tokens (   59.01 ms per token,    16.95 tokens per second)
llama_print_timings:        eval time =    3532.20 ms /    22 runs   (  160.55 ms per token,     6.23 tokens per second)
llama_print_timings:       total time =    4282.23 ms /    34 tokens


 I now know the final answer to the original question.
Final Answer: {'response': 'Post opened'}

> Finished chain.


In [29]:
print(A)

{'response': 'Post opened'}
